In [37]:
# load packages
import os,glob,sys
import xesmf as xe
import xarray as xr
import numpy as np
from netCDF4 import Dataset
import pandas as pd
import cartopy.crs as crs
import matplotlib
from cartopy.feature import NaturalEarthFeature 
import cartopy.feature as cfeature
import datetime
from wrf import (getvar, interplevel, vertcross, 
                 CoordPair, ALL_TIMES, to_np,
                 get_cartopy, latlon_coords,
                 cartopy_xlim, cartopy_ylim,
                 Constants,extract_vars)
from matplotlib import pyplot as plt
# Define global setting
xr.set_options(keep_attrs=True)

In [2]:
def get_nth_word_custom_delimiter(string, delimiter, n):
    """
    Function: break strings by delimiter and grab nth element
    
    Input: full parameter of simulations
    
    Output: nth element in the parameter
    """
# Split string by delimiter
    words = string.split(delimiter)
# Grab nth element in the string
    if 1 <= n <= len(words):
        return words[n-1]
    else:
        return "Invalid value of N."

In [3]:
# Build curvilinear grids for netcdfs
def build_wrf_gird(WRF_d,geo_file):
    """
    Function: build x,y curvilinear grids from WRF input dataset and geo_em
    Input: path for WRF dataset, path for geo_em file
    Output: WRF netcdf with curvilinear grid in lat/lon, coordinate system for WRF24
    """
    # Open geo_em file, get HGT var, cart_proj and lats and lons
    geo=Dataset(geo_file)
    HGT = getvar(geo,"HGT_M",timeidx=ALL_TIMES)
    WRF_cart_proj = get_cartopy(HGT)
    WRF_lats, WRF_lons = latlon_coords(HGT)
    # Create WRF Projection
    wrf_globe = crs.Globe(ellipse=None,
                          semimajor_axis=Constants.WRF_EARTH_RADIUS,
                          semiminor_axis=Constants.WRF_EARTH_RADIUS)
    # Define a latitude/longitude coordinate system
    wrf_xform_crs = crs.Geodetic(globe=wrf_globe)
    # Open dataset
    if '_mon_' in WRF_d: 
         WRF=xr.open_dataset(WRF_d,decode_times=False,chunks={'month':12})
    elif '_sea_' in WRF_d:
        WRF=xr.open_dataset(WRF_d,decode_times=False,chunks={'season':4})
#     WRF=WRF_d
    # Insert lat and lon for re-gridding
    WRF_latlon=WRF.assign_coords({'lat':(('south_north','west_east'),WRF_lats.values),'lon':(('south_north','west_east'),WRF_lons.values)})
    # Drop coordinate XTIME
#     WRF_latlon=WRF_latlon.drop_vars(['XTIME'])
    return WRF_latlon,WRF_cart_proj, wrf_xform_crs

In [34]:
# Interpolate to WRF24 grids
def to_WRF_grid(WRF,WRF_proj,WRF_crs,obs_d=None):
    """
    Function: Interpolate input dataset to WRF24 grid and perform remapping 
    Input: WRF xarray dataset with lat and lon,WRF porjection, WRF coordinate system, 
           Observation dataset
           remap from polar to WRF24 for i.ie. Rutgers Northern Hemisphere 24 km Weekly Snow Cover Extent
           or from WGS1984 to WRF24 for i.e. ERA5 Land
    Output: netcdf with curvilinear grid in EPSG 4326
    """
    # Grab parameters from WRF dataset
    year_start=int(WRF.attrs['begin_date'][0:4])                          # Begin year
    year_end=int(WRF.attrs['end_date'][0:4])                              # End year
    
    # For WRF output
    if obs_d==None:
        # Generate lat long based on WRF Projection
        xform_pts = WRF_proj.transform_points(WRF_crs,to_np(WRF.lon),to_np(WRF.lat))
        WRF_x = xform_pts[...,0]
        WRF_y = xform_pts[...,1]
        # insert lat and lon grids into dataset         
        WRF_output=WRF.assign_coords({'lat':(('south_north','west_east'),WRF_y),'lon':(('south_north','west_east'),WRF_x)})
        # build file name
        wrf_cat=get_nth_word_custom_delimiter(WRF.attrs['description'],' ',1) # WRF category 
        freq=list(WRF.indexes)[0][0:3]                                        # Temporal frequency
        par_full=WRF.attrs['experiment']
        force_d=get_nth_word_custom_delimiter(par_full,'_',1)                 # Forcing dataset
        scen=get_nth_word_custom_delimiter(par_full,'_',2)                    # Scenario
        phys=get_nth_word_custom_delimiter(par_full,'_',4)                    # Physical configuration
        grid='wrf24'                                                          # Grid
        out_path=force_d+'/'+grid+'/'+phys+'/'
        if os.path.exists(out_path)==False:
            os.makedirs(out_path)
        else:
            print('Directory exists\n')
        outfile=os.path.join(out_path,'wrfplev3d_'+scen+'_'+freq+'-norm-'+str(year_start)+'-'+str(year_end)+'.nc')
        if os.path.isfile(outfile):
            print('File exists\n')
        else:
            WRF_output.to_netcdf(outfile)
        return WRF_output
    else:
    # Open dataset
    # For rutger dataset
        if 'rutgers' in obs_d:
            obs_raw=xr.open_dataset(obs_d,chunks={'time':100})
            
    # For ERA5-Land dataset
        elif 'era5l' in obs_d:
            obs_raw=xr.open_mfdataset(obs_d).sortby('time')
            # Rechunk dataset
            obs_raw=obs_raw.chunk({'time':200,'latitude':200,'longitude':500})
     #  For ERA5-Land dataset
        elif 'era5pl' in obs_d:
            obs_raw=xr.open_mfdataset(obs_d).sortby('time')
            # Rechunk dataset
            obs_raw=obs_raw.chunk({'time':60,'latitude':200,'longitude':200,'level':-1})
    # For CERES dataset
        elif 'CERES' in obs_d:
            obs_raw=xr.open_mfdataset(obs_d).sortby('time')
            # Rechunk dataset
            obs_raw=obs_raw.chunk({'time':100,'lon':200,'lat':88})
         # Rename lat/lon
        if ('rutgers' in obs_d) or ('era5l' in obs_d):
            obs_raw=obs_raw.rename({'latitude':'lat','longitude':'lon'})
        # Generate normal for dataset
        obs_norm=obs_raw.isel(time=(obs_raw.time.dt.date<datetime.date(2015+1,1,1))).groupby('time.month').mean('time')
#         obs_norm=obs_raw.groupby('time.month').mean('time')
#       obs_norm=obs_raw.isel(time=((obs_raw.time.dt.date>=datetime.date(2000,3,1))&(obs_raw.time.dt.date<=datetime.date(2014,12,1)))).groupby('time.season').mean('time')
        # Mask missing value for Rutger dataset
        if 'rutgers' in obs_d:
            # Extract lat/lon as 2d numpy arrays
            obs_lat=obs_norm.lat.values
            obs_lon=obs_norm.lon.values.T
            # Replace missing values with nan
            new_lat=np.where(obs_lat>90,np.nan,obs_lat)
            new_lon=np.where(obs_lon>180,np.nan,obs_lon)
            obs_norm=obs_norm.assign_coords({'lat':(('x','y'),new_lat),'lon':(('x','y'),new_lon)})
        # Regrid to WRF24
        Regridder=xe.Regridder(obs_norm,WRF,'patch')
        print('Regridding start')
        obs_output_re=Regridder(obs_norm,keep_attrs=True)    #Keep attributes
        # Transform to WRF Projection
        xform_pts = WRF_proj.transform_points(WRF_crs,to_np(obs_output_re.lon.values),to_np(obs_output_re.lat.values))
        obs_wrf_x = xform_pts[...,0]
        obs_wrf_y = xform_pts[...,1]
        obs_output_re=obs_output_re.assign_coords({'lat':(('south_north','west_east'),obs_wrf_y),
                                                        'lon':(('south_north','west_east',),obs_wrf_x)})
        return obs_output_re

In [5]:
# Compute biases and generate plots
def find_outlier(obs_d,conus_d,new_d,newloc_d,geo_file,v_min,v_max,obs_src=None,par=None):
    """
    Function: Compute biases and generate plot based on parameter chosen
    Inputs: Observational, WRF-Conus, New, newloc xarray datasets regridded to WRF24
            Observational dataset source (i.e. ERA5L/Rutger)
            parameter to compare (i.e. snow cover)
            Land mask as 2d numpy array
    Output: Bias plots
    """
    # Define seasons
    seasons=['DJF','MAM','JJA','SON'] 
    # Choose variable from xarray datasets
    if par=='SNOWC':
        if obs_src=='ERA5L':
            obs_raw=obs_d.snowc/100
        elif obs_src=='Rutger':
            obs_raw=obs_d.snow_cover_extent
        conus_raw=conus_d.SNOWC
        new_raw=new_d.SNOWC
        newloc_raw=newloc_d.SNOWC
    elif par=='SW_D':
        if obs_src=='ERA5L':
            obs_raw=obs_d.ssrd/86400 # convert J/m^2 to W/m^-2
        elif obs_src=='CERES':
            obs_raw=obs_d.sfc_sw_down_all_mon
        conus_raw=conus_d.ACSWDNB
        new_raw=new_d.ACSWDNB
        newloc_raw=newloc_d.ACSWDNB
    elif par=='SW_U':
        if obs_src=='ERA5L':
            obs_raw=(obs_d.ssrd-obs_d.ssr)/86400 # convert J/m^2 to W/
        elif obs_src=='CERES':
            obs_raw=obs_d.sfc_sw_up_all_mon
        conus_raw=conus_d.ACSWUPB
        new_raw=new_d.ACSWUPB
        newloc_raw=newloc_d.ACSWUPB
    elif par=='ALB':
        if obs_src=='ERA5L':
            obs_raw=(obs_d.ssrd-obs_d.ssr)/obs_d.ssrd/86400
        elif obs_src=='CERES':
            obs_raw=obs_d.sfc_sw_up_all_mon/obs_d.sfc_sw_down_all_mon
        conus_raw=conus_d.ACSWUPB/conus_d.ACSWDNB
        new_raw=new_d.ACSWUPB/new_d.ACSWDNB
        newloc_raw=newloc_d.ACSWUPB/newloc_d.ACSWDNB
    # Store WRF as dictionary
    WRF_raw={'CONUS':conus_raw,'NEW':new_raw,'NEWLOC':newloc_raw}
    # Compute biases and store in new dictionary
    WRF_biases={}
    # Extract land mask (0: water and 1: for land)
    LM = xr.open_dataset(geo_file)
    LM = LM.LANDMASK.values[0]
    # get shape for empty array
    N,M=LM.shape
    # Creat empty for storing lower/upper quantile and  points with data
    WRF_with_data=np.empty([4,3])
    WRF_all_pts=np.empty([3,4,N,M])
    for j,physics in enumerate(WRF_raw.keys()):
        WRF_biases[physics]=WRF_raw[physics]-obs_raw
        # Find values for seasonal quantiles
        print('Calculating for',physics,'....\n')
        WRF_all_pts[j]=WRF_biases[physics].values
        for i,s in enumerate(seasons):
            WRF_all_pts[j,i][LM==0]=np.nan
            WRF_biases[physics][i]= WRF_all_pts[j,i]
            # Find the total number of points with data
            WRF_with_data[i,j]=np.sum(~(np.isnan(WRF_all_pts[j,i])))
            # Compute % of outliers for lower and upper bound
            WRF_outlier_L=np.nansum(WRF_all_pts[j,i]<v_min[i])/WRF_with_data[i,j]*100
            WRF_outlier_U=np.nansum(WRF_all_pts[j,i]>v_max[i])/WRF_with_data[i,j]*100
        # Print results
            print("{:<80}".format('% points outside range (lower,upper,total) for var '+par+':'+ \
                ', season '+s+', model '+physics+ \
                ' ='),"%.2f"%(WRF_outlier_L),"%.2f"%(WRF_outlier_U),"%.2f"%(WRF_outlier_L+WRF_outlier_U),'\n')
    return WRF_biases,WRF_all_pts

In [6]:
def build_plots(WRF_biases,WRF_proj,WRF_all_pts,v_min,v_max,obs=None,par=None,par_s=None):
    # Set variables for plots
    seasons=['DJF','MAM','JJA','SON'] 
    season_name=['Winter','Spring','Summer','Fall']
    # Set natural feature
    states = cfeature.NaturalEarthFeature(
        category='cultural',
        name='admin_1_states_provinces_lines',
        scale='50m',
        facecolor='none')
    land_edge_color=[0.55, 0.55, 0.55]
    # Replace inf with nan for calculation
    WRF_all_pts[WRF_all_pts==np.inf]=np.nan
    # Plotting
    fig = plt.figure(figsize=(28,42))
    # Add season titles
    for i,s in enumerate(seasons):
        # Add season name
        fig.text(-0.04,0.14+(3-i)*0.24,season_name[i], ha='center',va='center',rotation=90,fontsize=48)
        # Set conditions for colorbar if 0 is not in range
        if v_min[i]>=0:
            c_norm = matplotlib.colors.TwoSlopeNorm(vmin=-0.2,vcenter=0.0,vmax=v_max[i]) 
        elif v_max[i]<=0:
            c_norm = matplotlib.colors.TwoSlopeNorm(vmin=v_min[i],vcenter=0.0,vmax=0.2)
        else:
            c_norm = matplotlib.colors.TwoSlopeNorm(vmin=v_min[i],vcenter=0.0,vmax=v_max[i])
        for j,physics in enumerate(WRF_biases.keys()):
            # choose plot location and projection
            ax = fig.add_subplot(4,3,1+j+3*i,projection=WRF_proj)
            # Add Natural features
            ax.add_feature(cfeature.LAND,linewidth=1,edgecolor=land_edge_color,facecolor="none",zorder=3) 
            ax.add_feature(states,linewidth=1,edgecolor=land_edge_color,facecolor="none",zorder=3)   
            ax.add_feature(cfeature.LAKES,linewidth=1,edgecolor=land_edge_color,facecolor="none",zorder=3)
            ax.add_feature(cfeature.BORDERS,linewidth=1,edgecolor=land_edge_color,facecolor="none",zorder=3)
            # generate plots
            S=WRF_biases[physics].sel(season=s).plot.pcolormesh(x='lon',y='lat',ax=ax,       # x, y axes
                                                                add_colorbar=False,          # remove colorbar
                                                                cmap='bwr',                  # colors
                                                                norm=c_norm)                 # 
                                                                
            # Remove title
            ax.set_title('')

            # Calculate and show mean (mean over space) seasonal bias
            fig.text(0.03+0.335*j,0.783-i*0.24,"%.2f"%round(np.nanmean(np.abs(WRF_all_pts[j,i]),dtype=float),2), \
                ha='center',va='center',fontsize=40)   
            print(physics,s,'done')
        # Add colorbars
        cbar_ax = fig.add_axes([0.27,0.745-i*0.24,0.465,0.013])     
        cbar = plt.colorbar(S,cax=cbar_ax,orientation="horizontal",
                            extend='both') 
        tick_locator = matplotlib.ticker.MaxNLocator(nbins=5,
                                                    min_n_ticks=6,
                                                    symmetric=True)
        cbar.locator = tick_locator
        cbar.update_ticks()        
        cbar.ax.tick_params(labelsize=35)
        cbar.ax.set_xscale('linear')
    # Set figure layout and add x and y labels and titles, etc 
    fig.tight_layout(pad=0.0,w_pad=2.0,h_pad=0)     
    # Add overall plot title
    fig.text(0.5,1.02,par+' Average Seasonal Bias of WRF (2000-2014) as Compared to '+obs,ha='center',va='center',fontsize=40)
        
    # Add plot titles and sub-titles
    vx=[0.17,0.50,0.84]
    # title y location
    vy_t= 0.975
    # suptitle y location
    vy_supt = -0.01     
    for i, physics in enumerate(WRF_biases.keys()):
        fig.text(vx[i],vy_t,r'$\overline{({%s}_{%s}-{%s}_{Ob})}$' %(par_s,physics,par_s),
                 ha='center',va='center',fontsize=35)
        fig.text(vx[i],vy_supt,physics+' '+par_s+' Abs. Bias',ha='center',va='center',fontsize=48)